<a href="https://colab.research.google.com/github/SuminBae97/MachineLearning/blob/master/fbprophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>필요 패키지 로딩</h1>

In [ ]:
import pandas as pd
import pandas_datareader as web
import matplotlib.pyplot as plt
from fbprophet import Prophet
import  numpy as np

<h1>Prophet tutrial </h1>

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv("example_wp_log_peyton_manning.csv")

In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(df['y'],color='green')

In [ ]:
df.shape

<h1>Prophet model 호출</h1>

In [ ]:
model = Prophet()

<h1>Pass historical data</h1>

In [ ]:
model.fit(df)

In [ ]:
help(model.predict)

<h1>미래 날짜 data frame 제작 1년치</h>

In [ ]:
future = model.make_future_dataframe(periods=365)

In [ ]:
forecast = model.predict(future)

<h1>이전 기록을 포함을 할 수 있음</h1>

In [ ]:
help(model.make_future_dataframe)

In [ ]:
forecast.columns

In [ ]:
help(model.plot)

In [ ]:
fig1 = model.plot(forecast)

In [ ]:
forecast.shape

In [ ]:
help(model.plot_components)

In [ ]:
fig2 = model.plot_components(forecast)

<h1>기존 데이터 2007~2016</h1>

In [ ]:
forecast.columns

<h1>upper lower는 예측의 범위</h1>

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(forecast['yhat_lower'],color='red',label='yhat_lower')
plt.plot(forecast['yhat_upper'],color='blue',label='yhat_upper')
plt.plot(df['y'],color='green',label='ground truth')
plt.plot(forecast['yhat'],color='orange',label='yhat')
plt.legend()

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(df['y'],color='green',label='ground truth')
plt.plot(forecast['yhat'],color='orange',label='yhat')
plt.legend()

<h1>train validation으로 나눠서 학습</h1>

train : 2000개의 데이터로 추세 파악
validation : 600개의 데이터로 검증 

test : 300개의 데이터로 성능평가

In [ ]:
df.shape

In [ ]:
train_data = df.iloc[:2000]
validation_data = df.iloc[2000:2600]
test_data = df.iloc[2600:]

In [ ]:
train_data.shape, test_data.shape, validation_data.shape

In [ ]:
model2 = Prophet()
model2.fit(train_data)

In [ ]:
#forecast2 = model2.predict(test_data)

forecast2 = model2.predict(validation_data)

In [ ]:
forecast2.shape

In [ ]:
forecast2.columns

In [ ]:
#test_data_index_reset = test_data.reset_index()

validation_data = validation_data.reset_index()

In [ ]:
validation_data.head(5)

In [ ]:
validation_data.shape , forecast2.shape

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(validation_data['y'],label='ground_truth')
plt.plot(forecast2['yhat'],label='yhat')
# plt.plot(forecast2['yhat_upper'],label='yhat_upper')
# plt.plot(forecast2['yhat_lower'],label='yhat_lower')
plt.legend()

<h1>MAPE 계산</h1>

In [ ]:
def MAPE(y_test,y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

In [ ]:
MAPE(validation_data['y'],forecast2['yhat'])

In [ ]:
from fbprophet.plot import add_changepoints_to_plot

In [ ]:
fig1 = model2.plot(forecast2)
a = add_changepoints_to_plot(fig1.gca(),model2,forecast2)

In [ ]:
forecast2['ds'], validation_data['ds']

**parameter searching**
changepoint_range = 0.9 (default 0.8)
changepoint_prior_scale = 0.1(default 0.05) 값을 늘리면 underfitting 해결, 값을 줄이면 overfitting 해결



In [ ]:
model3 = Prophet(changepoint_range=0.9, changepoint_prior_scale=0.5)

In [ ]:
model3 = Prophet(changepoint_range=0.9, changepoint_prior_scale=0.001)
model3.fit(train_data)
forecast3 = model3.predict(validation_data)


In [ ]:
changepoint_prior_scale=[0.01,0.05,0.1,0.5,1]

for cps in changepoint_prior_scale:
    model = Prophet(changepoint_range=0.9, changepoint_prior_scale=cps)
    model.fit(train_data)
    forecast = model.predict(validation_data)
    plt.figure(figsize=(15,6))
    mape = MAPE(validation_data['y'],forecast['yhat'])
    plt.title("Model with changepoint_prior scale {} and mape: {}".format(cps,round(mape,2)),fontsize=25)
    plt.plot(validation_data['y'],label='ground_truth')
    plt.plot(forecast['yhat'],label='yhat')
    plt.legend()

